In [1]:
"""This script will use esm models to visualize embeddings of proteins from SKEMPI datasets"""
import torch
import esm
import pandas as pd
import scanpy as sc
import numpy as np

In [2]:
# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t48_15B_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

ESM2(
  (embed_tokens): Embedding(33, 5120, padding_idx=1)
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=5120, out_features=5120, bias=True)
        (v_proj): Linear(in_features=5120, out_features=5120, bias=True)
        (q_proj): Linear(in_features=5120, out_features=5120, bias=True)
        (out_proj): Linear(in_features=5120, out_features=5120, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=5120, out_features=20480, bias=True)
      (fc2): Linear(in_features=20480, out_features=5120, bias=True)
      (final_layer_norm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=5120, out_features=5120, bias=True)
        (v_proj): Linear(in_features=5120, out_features=5120, b

In [ ]:
print('test')